# Import libraries

In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


# Import and adjust data for analysis

## Minutes data adjustement

In [3]:
# Already downloande data from MT5
hdf = pd.read_csv('../Data/GBPUSD_M15_202001020000_202312292345.csv',sep='\t')

In [5]:
hdf

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,2020.01.02,00:00:00,1.32465,1.32479,1.32455,1.32467,94,0,24
1,2020.01.02,00:15:00,1.32467,1.32498,1.32460,1.32490,86,0,3
2,2020.01.02,00:30:00,1.32492,1.32512,1.32472,1.32491,826,0,1
3,2020.01.02,00:45:00,1.32491,1.32528,1.32476,1.32481,562,0,40
4,2020.01.02,01:00:00,1.32481,1.32609,1.32481,1.32567,382,0,8
...,...,...,...,...,...,...,...,...,...
99607,2023.12.29,22:45:00,1.27512,1.27517,1.27449,1.27479,541,0,2
99608,2023.12.29,23:00:00,1.27478,1.27480,1.27374,1.27382,308,0,2
99609,2023.12.29,23:15:00,1.27383,1.27388,1.27359,1.27383,180,0,2
99610,2023.12.29,23:30:00,1.27390,1.27412,1.27350,1.27350,322,0,2


In [6]:
# Def for clean data
def clean_df_minutes(df):
    df = df.copy()
    df['time'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    
    df.drop(['<DATE>','<TIME>','<VOL>','<SPREAD>'],axis=1,inplace=True)
    
    df = df[['time','<OPEN>','<HIGH>','<LOW>','<CLOSE>','<TICKVOL>']]
    df.columns = ['time','open','high','low','close','volume']
    
    df.set_index('time',inplace=True)
    
    return df

In [7]:
hdf = clean_df_minutes(hdf)


In [8]:
# Control data preparation
hdf

,open,high,low,close,volume
time,,,,,
2020-01-02 00:00:00,1.32465,1.32479,1.32455,1.32467,94
2020-01-02 00:15:00,1.32467,1.32498,1.32460,1.32490,86
2020-01-02 00:30:00,1.32492,1.32512,1.32472,1.32491,826
2020-01-02 00:45:00,1.32491,1.32528,1.32476,1.32481,562
2020-01-02 01:00:00,1.32481,1.32609,1.32481,1.32567,382
...,...,...,...,...,...
2023-12-29 22:45:00,1.27512,1.27517,1.27449,1.27479,541
2023-12-29 23:00:00,1.27478,1.27480,1.27374,1.27382,308
2023-12-29 23:15:00,1.27383,1.27388,1.27359,1.27383,180


In [9]:
# Already downloande data from MT5
ldf = pd.read_csv('../Data/GBPUSD_M1_202001020005_202312191921.csv',sep='\t')

In [10]:
# Def for clean data
def clean_df_minutes(df):
    df = df.copy()
    df['time'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    
    df.drop(['<DATE>','<TIME>','<TICKVOL>','<VOL>','<SPREAD>'],axis=1,inplace=True)
    
    df = df[['time','<OPEN>','<HIGH>','<LOW>','<CLOSE>']]
    df.columns = ['time','open','high','low','close']
    
    df.set_index('time',inplace=True)
    
    return df

In [11]:
ldf = clean_df_minutes(ldf)


In [12]:
# Control data preparation
ldf.head()

,open,high,low,close
time,,,,
2020-01-02 00:05:00,1.32465,1.32465,1.32455,1.32464
2020-01-02 00:06:00,1.32464,1.32471,1.32462,1.32471
2020-01-02 00:07:00,1.32471,1.32471,1.32462,1.32465
2020-01-02 00:08:00,1.32465,1.32465,1.32462,1.32465
2020-01-02 00:09:00,1.32465,1.32465,1.32462,1.32464


## Tick data adjustement

In [3]:
df = pd.read_csv('../Data/EURUSD_202302081525_202401021452.csv', sep = '\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14229600 entries, 0 to 14229599
Data columns (total 7 columns):
 #   Column    Dtype  
---  ------    -----  
 0   <DATE>    object 
 1   <TIME>    object 
 2   <BID>     float64
 3   <ASK>     float64
 4   <LAST>    float64
 5   <VOLUME>  float64
 6   <FLAGS>   int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 759.9+ MB


In [4]:
def clean_df_ticks(tick_data):
    # Make a copy to avoid the SettingWithCopyWarning
    tick_data = tick_data.copy()
    
    # Convert to datetime
    tick_data['time'] = pd.to_datetime(tick_data['<DATE>'] + ' ' + tick_data['<TIME>'])
    
    # Drop unnecessary columns
    tick_data.drop(['<LAST>', '<VOLUME>', '<FLAGS>', '<DATE>', '<TIME>'], axis=1, inplace=True)
    
    # Rearrange columns
    tick_data = tick_data[['time', '<BID>', '<ASK>']]
    tick_data.columns = ['time', 'bid', 'ask']
    
    # Forward-fill missing values in 'bid' and 'ask' columns
    tick_data['bid'].ffill(axis=0, inplace=True)
    tick_data['ask'].ffill(axis=0, inplace=True)
    
    # Set 'time' column as index
    tick_data.set_index('time', inplace=True)
    
    return tick_data

In [5]:
df = clean_df_ticks(df)

In [6]:
# Control data preparation
df.head()

,bid,ask
time,,
2023-02-08 15:25:00.924,1.07371,1.07401
2023-02-08 15:25:01.964,1.07372,1.07402
2023-02-08 15:25:03.034,1.07371,1.07401
2023-02-08 15:25:03.212,1.07372,1.07402
2023-02-08 15:25:03.470,1.07373,1.07403


In [8]:
df.to_csv('../ReadyData/EURUSD_2023_tick.csv')

# High Low time adjustement from lower tf

In [13]:

def find_timestamp_extremum(df, df_lower_timeframe):
    """
    :param: df_lowest_timeframe
    :return: self._data with three new columns: Low_time (TimeStamp), High_time (TimeStamp), High_first (Boolean)
    """
    df = df.copy()
    df = df.loc[df_lower_timeframe.index[0]:]

    # Set new columns
    df["low_time"] = np.nan
    df["high_time"] = np.nan

    # Loop to find out which of the high or low appears first
    for i in tqdm(range(len(df) - 1)):

        # Extract values from the lowest timeframe dataframe
        start = df.iloc[i:i + 1].index[0]
        end = df.iloc[i + 1:i + 2].index[0]
        row_lowest_timeframe = df_lower_timeframe.loc[start:end].iloc[:-1]

        # Extract Timestamp of the max and min over the period (highest timeframe)
        try:
            high = row_lowest_timeframe["high"].idxmax()
            low = row_lowest_timeframe["low"].idxmin()

            df.loc[start, "low_time"] = low
            df.loc[start, "high_time"] = high

        except Exception as e:
            print(e)
            df.loc[start, "low_time"] = None
            df.loc[start, "high_time"] = None

    # Verify the number of row without both TP and SL on same time
    percentage_garbage_row = len(df.dropna()) / len(df) * 100

    # if percentage_garbage_row<95:
    print(f"WARNINGS: Garbage row: {'%.2f' % percentage_garbage_row} %")

    df = df.iloc[:-1]

    return df


In [14]:

#df_low_tf = pd.read_csv("FixTimeBars/AUDUSD_30M_Admiral.csv", index_col="time", parse_dates=True)
#df_high_tf = pd.read_csv("FixTimeBars/AUDUSD_4H_Admiral.csv", index_col="time", parse_dates=True)

df = find_timestamp_extremum(hdf, ldf)

print(df[["high_time", "low_time"]])


  0%|          | 0/99610 [00:00<?, ?it/s]

  5%|▍         | 4630/99610 [00:01<00:36, 2585.01it/s]

attempt to get argmax of an empty sequence


 74%|███████▍  | 73756/99610 [00:29<00:10, 2541.54it/s]

attempt to get argmax of an empty sequence


...

attempt to get argmax of an empty sequence


100%|█████████▉| 99326/99610 [00:40<00:00, 2271.79it/s]

attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to 

100%|██████████| 99610/99610 [00:40<00:00, 2461.89it/s]

attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to 

In [15]:
df.to_csv("../ReadyData/GBPUSD_2020_2023_15M_READY.csv")


In [11]:

def find_timestamp_extremum_tick(df, df_lower_timeframe):
    """
    :param: df_lowest_timeframe
    :return: self._data with three new columns: Low_time (TimeStamp), High_time (TimeStamp), High_first (Boolean)
    """
    df = df.copy()
    df = df.loc[df_lower_timeframe.index[0]:]

    # Set new columns
    df["low_time"] = np.nan
    df["high_time"] = np.nan

    # Loop to find out which of the high or low appears first
    for i in tqdm(range(len(df) - 1)):

        # Extract values from the lowest timeframe dataframe
        start = df.iloc[i:i + 1].index[0]
        end = df.iloc[i + 1:i + 2].index[0]
        row_lowest_timeframe = df_lower_timeframe.loc[start:end].iloc[:-1]

        # Extract Timestamp of the max and min over the period (highest timeframe)
        try:
            high = row_lowest_timeframe["ask"].idxmax()
            low = row_lowest_timeframe["bid"].idxmin()

            df.loc[start, "low_time"] = low
            df.loc[start, "high_time"] = high

        except Exception as e:
            print(e)
            df.loc[start, "low_time"] = None
            df.loc[start, "high_time"] = None

    # Verify the number of row without both TP and SL on same time
    percentage_garbage_row = len(df.dropna()) / len(df) * 100

    # if percentage_garbage_row<95:
    print(f"WARNINGS: Garbage row: {'%.2f' % percentage_garbage_row} %")

    df = df.iloc[:-1]

    return df


#df_low_tf = pd.read_csv("FixTimeBars/AUDUSD_30M_Admiral.csv", index_col="time", parse_dates=True)
#df_high_tf = pd.read_csv("FixTimeBars/AUDUSD_4H_Admiral.csv", index_col="time", parse_dates=True)


In [19]:
df = find_timestamp_extremum_tick(hdf, df)


  0%|          | 0/72892 [00:00<?, ?it/s]

C:\Users\jakub\AppData\Local\Temp\ipykernel_20232\229157434.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-01-02 00:14:09.198000' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[start, "low_time"] = low
C:\Users\jakub\AppData\Local\Temp\ipykernel_20232\229157434.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-01-02 00:13:35.907000' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[start, "high_time"] = high
  1%|          | 371/72892 [00:05<09:52, 122.42it/s]

attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence


  4%|▎         | 2684/72892 [00:06<00:35, 1986.06it/s]

attempt to get argmax of an empty sequence


...

attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to 

100%|█████████▉| 72877/72892 [00:56<00:00, 1266.42it/s]

attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to 

100%|██████████| 72892/72892 [00:56<00:00, 1288.32it/s]

attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
attempt to get argmax of an empty sequence
WARNINGS: Garbage row: 97.01 %


# Saving Prepared Data

Using pickle is particularly efficient for large-sized tick data.

Pickle preserves the indexing, eliminating the need for re-indexing during subsequent loads.

In [20]:
#Save as compressed
#df.to_pickle('USDCHF_2020_2023_4H.bz2')
df.to_csv('ReadyData/EURUSD_2023_5m.csv')

# Load ready data

In [21]:
df

,open,high,low,close,low_time,high_time
time,,,,,,
2023-01-02 00:10:00,1.07024,1.07073,1.06948,1.06956,2023-01-02 00:14:09.198000,2023-01-02 00:13:35.907000
2023-01-02 00:15:00,1.06938,1.06956,1.06931,1.06955,2023-01-02 00:17:30.711000,2023-01-02 00:15:24.102000
2023-01-02 00:20:00,1.06957,1.06959,1.06957,1.06957,2023-01-02 00:20:02.041000,2023-01-02 00:21:08.641000
2023-01-02 00:25:00,1.06956,1.06957,1.06956,1.06957,2023-01-02 00:26:33.797000,2023-01-02 00:27:28.799000
2023-01-02 00:30:00,1.06962,1.06962,1.06936,1.06938,2023-01-02 00:31:00.705000,2023-01-02 00:30:00.509000
...,...,...,...,...,...,...
2023-12-22 23:00:00,1.10118,1.10123,1.10113,1.10115,None,None
2023-12-22 23:05:00,1.10114,1.10122,1.10107,1.10117,None,None
2023-12-22 23:10:00,1.10115,1.10122,1.10101,1.10106,None,None


In [ ]:
df = pd.read_pickle('EURUSD_2020-2023_tick.bz2')